In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import time
import tensorflow as tf
import p8_util
import p8_util_config

Using TensorFlow backend.


In [3]:
import p8_util
datadir = './data'

filename_dataset=datadir+'/arr_keras_X_y_train_test.dump'
x_train, x_test, y_train, y_test, nClasses, feature_shape = p8_util.load_dataset(filename_dataset,is_label_encoded=True)

print("\nDataset : {}".format((x_train.shape, x_test.shape, y_train.shape, y_test.shape, nClasses, feature_shape)))

p5_util.object_load : fileName= ./data/arr_keras_X_y_train_test.dump
(414, 224, 224, 3) (47, 224, 224, 3) (414, 3) (47, 3)

Dataset : ((414, 224, 224, 3), (47, 224, 224, 3), (414,), (47,), 3, (224, 224, 3))


In [4]:
OUTPUT_DIR = './tmp/baseline'
OUTPUT_DIR_TB = './tmp'
is_tensorboard = True

#### Tensorboard processes are killed

In [5]:
# this will kill the processes for Tensorboard
#is_tensorboard = True
if is_tensorboard is True :
    !ps aux | grep tensorboard | awk '{print $2}' | xargs kill
# this will kill the processes for ngrok
if is_tensorboard is True :
    !ps aux | grep ngrok | awk '{print $2}' | xargs kill    

kill: (5055)kill: (5061)

#### Increase of train dataset against test dataset.

In [6]:
OUTPUT_DIR

'./tmp/baseline'

In [7]:
import p8_util
print("Input output dir= {}".format(OUTPUT_DIR))
oNNAdaNetBuilder = p8_util.create_nn_builder(OUTPUT_DIR)
oNNAdaNetBuilder.show()

Input output dir= ./tmp/baseline


Global steps  : ................................... 5
NN type              : ............................ DNN
Features shape       : ............................ (224, 672)
Adanet outputdir     : ............................ ./tmp/baseline/DNN
Adanet output log    : ............................ ./tmp/baseline/DNN/DNN
Adanet boosting iter.: ............................ 1
Adanet iter per boost: ............................ 5
Number of layers     : ............................ 6
Dropout rate         : ............................ 0.0
Seed value           : ............................ 42
Nb of classes (logit): ............................ 3
Adanet regularization: ............................ 1e-05
Weights initializer  : ............................ xavier
Batch normalization  : ............................ True
Learn mixture weights: ............................ True


Number of layers     : ............................ 6
Hidden units         : .........

#### Customized Tensor Flow Estimator

In [8]:
oNNAdaNetBuilder = p8_util.create_nn_builder(OUTPUT_DIR)
oNNAdaNetBuilder.show()
params = {'net_builder': oNNAdaNetBuilder}



Global steps  : ................................... 5
NN type              : ............................ DNN
Features shape       : ............................ (224, 672)
Adanet outputdir     : ............................ ./tmp/baseline/DNN
Adanet output log    : ............................ ./tmp/baseline/DNN/DNN
Adanet boosting iter.: ............................ 1
Adanet iter per boost: ............................ 5
Number of layers     : ............................ 6
Dropout rate         : ............................ 0.0
Seed value           : ............................ 42
Nb of classes (logit): ............................ 3
Adanet regularization: ............................ 1e-05
Weights initializer  : ............................ xavier
Batch normalization  : ............................ True
Learn mixture weights: ............................ True


Number of layers     : ............................ 6
Hidden units         : ............................ 128


#### Create a Baseline Estimator

In [9]:
import BaselineEstimator
oBaselineEstimator = BaselineEstimator.BaselineEstimator( params,x_train, x_test, y_train, y_test, nClasses, feature_shape)

INFO:tensorflow:Using config: {'_model_dir': './tmp/baseline/DNN/DNN', '_tf_random_seed': 42, '_save_summary_steps': 10, '_save_checkpoints_steps': 10, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f43ce654c18>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


I0521 23:08:38.845124 139931040925504 estimator.py:201] Using config: {'_model_dir': './tmp/baseline/DNN/DNN', '_tf_random_seed': 42, '_save_summary_steps': 10, '_save_checkpoints_steps': 10, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f43ce654c18>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [10]:
results, start_time, end_time  = oBaselineEstimator.train_and_evaluate()

INFO:tensorflow:Not using Distribute Coordinator.


I0521 23:08:38.929777 139931040925504 estimator_training.py:185] Not using Distribute Coordinator.


INFO:tensorflow:Running training and evaluation locally (non-distributed).


I0521 23:08:38.931322 139931040925504 training.py:610] Running training and evaluation locally (non-distributed).


INFO:tensorflow:Start train and evaluate loop. The evaluate will happen after every checkpoint. Checkpoint frequency is determined based on RunConfig arguments: save_checkpoints_steps 10 or save_checkpoints_secs None.


...

I0521 23:08:42.657863 139931040925504 estimator.py:1979] Saving dict for global step 3: DNN_Eval_accuracy = 0.31914893, global_step = 3, loss = 1.1001211


INFO:tensorflow:Saving 'checkpoint_path' summary for global step 3: ./tmp/baseline/DNN/DNN/model.ckpt-3


I0521 23:08:42.709288 139931040925504 estimator.py:2039] Saving 'checkpoint_path' summary for global step 3: ./tmp/baseline/DNN/DNN/model.ckpt-3


INFO:tensorflow:Loss for final step: 1.0980539.


I0521 23:08:42.752638 139931040925504 estimator.py:359] Loss for final step: 1.0980539.


In [11]:
print("\n------------------------------------------------")
print("Time (sec)", end_time-start_time)
for key in results.keys() :
    title =key.upper()
    print("{}: {}".format(title, results[key]))
print("------------------------------------------------\n")



------------------------------------------------
Time (sec) 3.82422137260437
DNN_EVAL_ACCURACY: 0.3191489279270172
LOSS: 1.1001211404800415
GLOBAL_STEP: 3
------------------------------------------------



In [12]:
output_dir_log = './tmp/baseline/RNN/RNN'

In [13]:
oNNAdaNetBuilder.output_dir_log, is_tensorboard
output_dir_log = oNNAdaNetBuilder.output_dir_log
output_dir_log

'./tmp/baseline/DNN/DNN'

In [14]:
if is_tensorboard is True :
    get_ipython().system_raw(
        'tensorboard --logdir {} --host localhost --port 6007 &'
        .format(output_dir_log)
    )


    get_ipython().system_raw('./assets/ngrok http 6007 &')

In [15]:
is_tensorboard,output_dir_log

(True, './tmp/baseline/DNN/DNN')